#### Imports

In [10]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot

#### Discriminator Model

In [11]:
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1,activation="sigmoid"))

    opt = Adam(lr=0.0002,beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

#### Generator Model

In [12]:
def define_generator(latent_dim):
    model = Sequential()
    n_node = 128 *7 *7
    model.add(Dense(n_node, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7,7,128)))

    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Conv2D(1, (7,7), activation="sigmoid", padding="same"))
    return model

#### Gans Function

In [13]:
def define_gan(g_model, d_model):
    d_model.trainable = False

    model = Sequential()
    model.add(g_model)
    model.add(d_model)

    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

#### Helper Functions

In [19]:
# load and prepare mnist training images
def load_real_samples():
    (trainX,_),(_,_) = load_data()
    X = expand_dims(trainX, axis = -1)
    X=X.astype('float32')
    X=X/255.0
    return X

# select real samples
def generate_real_samples(dataset, n_samples):
    ix = randint(0, dataset.shape[0], n_samples)
    X = dataset[ix]
    y = ones((n_samples,1))
    return X,y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,n_samples):
    x_input = randn(latent_dim * n_samples)
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input

# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model,latent_dim,n_samples):
    x_input = generate_latent_points(latent_dim,n_samples)
    X = g_model.predict(x_input)
    y = zeros((n_samples,1))
    return X,y

# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    for i in range(n * n):
        pyplot.subplot(n,n,1+i)
        pyplot.axis('off')
        pyplot.imshow(examples[i,:,:,0], cmap='gray_r')
    
    filename = 'generated_plot_ep_%03d.png'%(epoch+1)
    pyplot.savegif(filename)
    pyplot.close()

# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    X_real, y_real = generate_real_samples(dataset, n_samples)
    _, acc_real = d_model.evaluate(X_real,y_real, verbose=0)
    X_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    _, acc_fake = d_model.evaluate(X_fake, y_fake, verbose=0)

    print("Accuracy real: %.0f%%, fake: %.0f%%" % (acc_real*100, acc_fake*100))
    save_plot(X_fake, epoch)

    filename = 'generator_model_%03d.h5' %(epoch+1)
    g_model.save(filename)

#### Training Function

In [24]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=100, n_batch=256):
    bat_per_epoch = int(dataset.shape[0]/n_batch)
    half_batch = int(n_batch/2)

    for i in range(n_epochs):
        for j in range(bat_per_epoch):

            X_real, y_real = generate_real_samples(dataset, half_batch)
            X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            X,y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
            d_loss,_ = d_model.train_on_batch(X,y)

            X_gan = generate_latent_points(latent_dim, n_batch)
            y_gan = ones((n_batch,1))

            g_loss = gan_model.train_on_batch(X_gan, y_gan)

            print(">epoch%d, %d/%d, d_loss=%.3f, g_loss=%.3f" %(i+1,j+1,bat_per_epoch,d_loss,g_loss))
        
        if(i+1) % 10 ==10:
            summarize_performance(i, g_model, d_model, dataset, latent_dim)
    

In [25]:
latent_dim = 100
d_model = define_discriminator()
g_model = define_generator(latent_dim)
gan_model = define_gan(g_model, d_model)
dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim)


C:\Users\pooja\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


>epoch1, 1/234, d_loss=0.713, g_loss=0.676
>epoch1, 2/234, d_loss=0.704, g_loss=0.701
>epoch1, 3/234, d_loss=0.691, g_loss=0.715
>epoch1, 4/234, d_loss=0.684, g_loss=0.738
>epoch1, 5/234, d_loss=0.677, g_loss=0.755
>epoch1, 6/234, d_loss=0.670, g_loss=0.778
>epoch1, 7/234, d_loss=0.656, g_loss=0.799
>epoch1, 8/234, d_loss=0.650, g_loss=0.810
>epoch1, 9/234, d_loss=0.648, g_loss=0.823
>epoch1, 10/234, d_loss=0.641, g_loss=0.833
>epoch1, 11/234, d_loss=0.641, g_loss=0.833
>epoch1, 12/234, d_loss=0.636, g_loss=0.828
>epoch1, 13/234, d_loss=0.639, g_loss=0.812
>epoch1, 14/234, d_loss=0.644, g_loss=0.800
>epoch1, 15/234, d_loss=0.645, g_loss=0.784
>epoch1, 16/234, d_loss=0.643, g_loss=0.769
>epoch1, 17/234, d_loss=0.643, g_loss=0.754
>epoch1, 18/234, d_loss=0.642, g_loss=0.740
>epoch1, 19/234, d_loss=0.639, g_loss=0.729
>epoch1, 20/234, d_loss=0.637, g_loss=0.718
>epoch1, 21/234, d_loss=0.630, g_loss=0.711
>epoch1, 22/234, d_loss=0.626, g_loss=0.705
>epoch1, 23/234, d_loss=0.617, g_loss=0.7